In [ ]:
%pip install openai # Install OpenAI API client
%pip install python-dotenv # Install python-dotenv for loading environment variables

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")
from os import getenv

key = getenv("DEEPSEEK_API_KEY") 
url = getenv("DEEPSEEK_API_BASE")
model = getenv("DEEPSEEK_MODEL")

from openai import OpenAI
client = OpenAI(api_key=key, base_url=url)
print(client.models.list())

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")
from os import getenv

key = getenv("DEEPSEEK_API_KEY") 
url = getenv("DEEPSEEK_API_BASE")
model = getenv("DEEPSEEK_MODEL")
rmodel = getenv("DEEPSEEK_RMODEL")

print(key, url, model, rmodel)


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=key, base_url=url)

response = client.chat.completions.create(
    model=rmodel,
    messages=[
        {"role": "system", "content": "你是经验丰富的c++程序员，擅长数据结构和算法"},
        {"role": "user", "content": "如何学习动态规划算法？"},
  ],
  max_tokens=1
)

print(response.choices[0].reasoning_content)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=key, base_url=url)
# Round 1
messages = [{"role": "user", "content": 
             "What's the highest mountain in the world?"}]

response = client.chat.completions.create(
    model=model,
    messages=messages
)
print("Round 1: \n", response.choices[0].message.content)

# Append the response to the messages list for the next round 
messages.append(response.choices[0].message)

# Round 2
messages.append({"role": "user", "content": "What is the second?"})
response = client.chat.completions.create(
    model=model,
    messages=messages
)

print("Round 2: \n", response.choices[0].message.content)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=key, base_url=url)

messages = [{"role": "user", "content": "解释量子计算"}]

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    stream=True,  # enable streaming
)

# process the response in chunks
for chunk in response:
    if chunk.choices[0].delta.content is not None: 
        print(chunk.choices[0].delta.content, end="")

In [ ]:
import json
from openai import OpenAI

client = OpenAI(api_key=key, base_url=url)

system_prompt = """
The user will provide some exam text. 
Please parse the "question" and "answer" and output them in JSON format. 

EXAMPLE INPUT: 
Which is the highest mountain in the world? Mount Everest.

EXAMPLE JSON OUTPUT:
{
    "question": "Which is the highest mountain in the world?",
    "answer": "Mount Everest"
}
"""

user_prompt = "Which is the longest river in the world? The Nile River."

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

response = client.chat.completions.create(
    model=model",
    messages=messages,
    response_format={
        'type': 'json_object'
    }
)

print(json.loads(response.choices[0].message.content))

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=key, base_url=url)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=[{"role": "user", "content": "What is the weather like in xiamen today?"}],
    tools=tools
)

print(completion.choices[0].message.tool_calls)